In this notebook we explore how well the algorithm is capable of recovering the following dependencies:
- Linear combination of 3 variables and bias with standard-normally distributed coefficients
  
  We can see that in 6 cases out of 10 the formulas are recovered exactly, while in other 5 cases there is a strong similarity (the degree of one variable is a small number, and the loss is compensated by the bias term).


- Linear combination of 3 squared variables and bias with standard-normally distributed coefficients

    We can clearly see that in 9 cases out of 10 the formulas are recovered exactly.


- Linear combination of 3 variables with powers uniformly distributed over {1, 2 ,3 ,4, 5} and standard-normally distributed coefficients

    We can see that in 6 cases out of 10 the formulas are recovered exactly, while in other 3 cases there is no similarity (the degree of one variable is a small number, and the loss is compensated by the bias term).

In [25]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from torch.nn import MSELoss
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import time
from IPython.display import display, Math, Latex

import NestedFormula
import auxiliary_functions
import explore
import importlib
from hessian import hessian
from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [45]:
importlib.reload(NestedFormula)
importlib.reload(auxiliary_functions)
importlib.reload(explore)

<module 'explore' from '/home/zybinmikhail/Documents/personal github projects/LearningFormulas/explore.py'>

## Linear combination of 3 variables and bias with standard-normally distributed coefficients

In [32]:
explore.explore()



----------------------------Exploring new formula #1----------------------------
  Initialization #1
    Epoch 5000, current loss 0.0449, current formula \left(-11.424x_1^{-0.027} + 10.731x_2^{-0.015} + 0.915x_3^{1.071} + 0.819\right)
  Finished run #1, loss 0.04426300898194313, best loss 0.04426300898194313
  Initialization #2
  Finished run #2, loss 0.011057988740503788, best loss 0.011057988740503788
  Initialization #3
  Finished run #3, loss 7.58524076882594e-10, best loss 7.58524076882594e-10
loss is smaller than 1e-05, terminating learning process
38 seconds passed from the start, the iteration took 38 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 1.2181853367313319e-08
EXACT RECOVERY


----------------------------Exploring new formula #2----------------------------
  Initialization #1
  Finished run #1, loss 0.01343916542828083, best loss 0.01343916542828083
  Initialization #2
  Finished run #2, loss 0.0015267726266756654, best loss 0.0015267726266756654
  Initialization #3
  Finished run #3, loss 0.013873125426471233, best loss 0.0015267726266756654
  Initialization #4
  Finished run #4, loss 0.011437230743467808, best loss 0.0015267726266756654
  Initialization #5
  Finished run #5, loss 0.00028848694637417793, best loss 0.00028848694637417793
  Initialization #6
  Finished run #6, loss 0.011361205950379372, best loss 0.00028848694637417793
  Initialization #7
  Finished run #7, loss 0.012482903897762299, best loss 0.00028848694637417793
  Initialization #8
    Epoch 5000, current loss 0.00934, current formula \left(5.719x_1^{-0.029}-0.210x_2^{1.102} + 1.330x_3^{-0.024}-6.305\right)
  Finish

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

MSE between formula parameters is 2.1145637650400927e-07
EXACT RECOVERY


----------------------------Exploring new formula #10----------------------------
  Initialization #1
  Finished run #1, loss 0.012512505054473877, best loss 0.012512505054473877
  Initialization #2
  Finished run #2, loss 0.011847813613712788, best loss 0.011847813613712788
  Initialization #3
  Finished run #3, loss 0.04677753150463104, best loss 0.011847813613712788
  Initialization #4
  Finished run #4, loss 2.9658236977425645e-10, best loss 2.9658236977425645e-10
loss is smaller than 1e-05, terminating learning process
3 minutes 6 seconds passed from the start, the iteration took 13 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 7.622096477786857e-09
EXACT RECOVERY

################################################################################
MSEs between parameters:
[1.2181853367313319e-08, 0.8217513293213506, 7.0111635002463e-07, 5.000738325021205e-08, 6.7070178810246794e-06, 3.073016812444556e-07, 3.166313272906507e-06, 1.9255488898599943e-07, 2.1145637650400927e-07, 7.622096477786857e-09]
For 9 formulas out of 10 the error is less than 1e-05.


## Linear combination of 3 squared variables and bias with standard-normally distributed coefficients

In [11]:
explore(min_power=2, max_power=2)



----------------------------Exploring new formula #1----------------------------
  Initialization #1
  Finished run #1, loss 0.19504092633724213, best loss 0.19504092633724213
  Initialization #2
  Finished run #2, loss 0.007013747934252024, best loss 0.007013747934252024
  Initialization #3
  Finished run #3, loss 0.009375261142849922, best loss 0.007013747934252024
  Initialization #4
  Finished run #4, loss 0.009800040163099766, best loss 0.007013747934252024
  Initialization #5
  Finished run #5, loss 1.8762023185070476e-12, best loss 1.8762023185070476e-12
loss is smaller than 0.001, terminating learning process
15 seconds passed from the start, the iteration took 15 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 6.69699216717033e-12
EXACT RECOVERY


----------------------------Exploring new formula #2----------------------------
  Initialization #1
  Finished run #1, loss 3.5127137743701242e-12, best loss 3.5127137743701242e-12
loss is smaller than 0.001, terminating learning process
16 seconds passed from the start, the iteration took 1 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

MSE between formula parameters is 7.782828465030533e-08
EXACT RECOVERY


----------------------------Exploring new formula #10----------------------------
  Initialization #1
  Finished run #1, loss 7.292567591465904e-12, best loss 7.292567591465904e-12
loss is smaller than 0.001, terminating learning process
2 minutes 55 seconds passed from the start, the iteration took 6 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 5.295644557788495e-10
EXACT RECOVERY

################################################################################
MSEs between parameters:
[6.69699216717033e-12, 7.412026548081485e-11, 3.558649565503304e-10, 4.6123866630815817e-10, 3.520428805612953e-09, 3.1134510842483516e-10, 6.36695601003672e-09, 2.306655452500273, 7.782828465030533e-08, 5.295644557788495e-10]
For 9 formulas out of 10 the error is less than 1e-05.


## Linear combination of 3 variables with powers uniformly distributed over {1, 2 ,3 ,4, 5} and standard-normally distributed coefficients

In [35]:
explore.explore(max_power=6)



----------------------------Exploring new formula #1----------------------------
  Initialization #1
  Finished run #1, loss 0.06304360181093216, best loss 0.06304360181093216
  Initialization #2
  Finished run #2, loss 0.004367060959339142, best loss 0.004367060959339142
  Initialization #3
  Finished run #3, loss 1.3601357019832339e-11, best loss 1.3601357019832339e-11
loss is smaller than 1e-05, terminating learning process
6 seconds passed from the start, the iteration took 6 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 5.667260058065235e-09
EXACT RECOVERY


----------------------------Exploring new formula #2----------------------------
  Initialization #1
  Finished run #1, loss 0.08279216289520264, best loss 0.08279216289520264
  Initialization #2
  Finished run #2, loss 1.7293509289514897e-12, best loss 1.7293509289514897e-12
loss is smaller than 1e-05, terminating learning process
10 seconds passed from the start, the iteration took 4 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

MSE between formula parameters is 2.363041794779213e-08
EXACT RECOVERY


----------------------------Exploring new formula #10----------------------------
  Initialization #1
  Finished run #1, loss 0.005140147637575865, best loss 0.005140147637575865
  Initialization #2
  Finished run #2, loss 1.1832983377868533e-11, best loss 1.1832983377868533e-11
loss is smaller than 1e-05, terminating learning process
3 minutes 6 seconds passed from the start, the iteration took 3 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 9.018604936760181e-07
EXACT RECOVERY

################################################################################
MSEs between parameters:
[5.667260058065235e-09, 1.1465926620855628e-10, 1.1858755247625855e-10, 3.3764445208005004e-10, 1.4469741119579486e-08, 1.5023411086596232e-10, 3.4865421660867923e-10, 1.1178741107390318e-08, 2.363041794779213e-08, 9.018604936760181e-07]
For 10 formulas out of 10 the error is less than 1e-05.


In [46]:
explore.explore(max_power=15, divide_powers_by=3)



----------------------------Exploring new formula #1----------------------------
  Initialization #1
  Finished run #1, loss 0.0005649023805744946, best loss 0.0005649023805744946
  Initialization #2
  Finished run #2, loss 3.363373165046757e-12, best loss 3.363373165046757e-12
loss is smaller than 1e-05, terminating learning process
3 seconds passed from the start, the iteration took 3 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 1.0837911044804726e-09
EXACT RECOVERY


----------------------------Exploring new formula #2----------------------------
  Initialization #1
  Finished run #1, loss 0.043854933232069016, best loss 0.043854933232069016
  Initialization #2
  Finished run #2, loss 5.613911113755421e-09, best loss 5.613911113755421e-09
loss is smaller than 1e-05, terminating learning process
15 seconds passed from the start, the iteration took 12 seconds
ground truth and obtained formula


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

MSE between formula parameters is 4.5763519957680336e-11
EXACT RECOVERY


----------------------------Exploring new formula #10----------------------------
  Initialization #1
  Finished run #1, loss 7.805804489180446e-05, best loss 7.805804489180446e-05
  Initialization #2
  Finished run #2, loss 0.04445670545101166, best loss 7.805804489180446e-05
  Initialization #3
  Finished run #3, loss 7.458134496118873e-05, best loss 7.458134496118873e-05
  Initialization #4
  Finished run #4, loss 6.3320629124064e-05, best loss 6.3320629124064e-05
  Initialization #5
  Finished run #5, loss 0.04257139191031456, best loss 6.3320629124064e-05
  Initialization #6
  Finished run #6, loss 0.04354075714945793, best loss 6.3320629124064e-05
  Initialization #7
  Finished run #7, loss 0.04352091625332832, best loss 6.3320629124064e-05
  Initialization #8
  Finished run #8, loss 6.833985389675945e-05, best loss 6.3320629124064e-05
  Initialization #9
    Epoch 5000, current loss 0.19, current formula \

<IPython.core.display.Math object>

<IPython.core.display.Math object>

MSE between formula parameters is 1.6850588041040635
FAILURE

################################################################################
MSEs between parameters:
[1.0837911044804726e-09, 7.233614190178222e-08, 2.464195771728083, 3.936620335265317e-08, 6.349257627594592e-11, 6.099755621237688e-11, 8.138962519816622e-11, 0.0006299513376686962, 4.5763519957680336e-11, 1.6850588041040635]
For 7 formulas out of 10 the error is less than 1e-05.
